# Your First RAG Application: Personal Wellness Assistant

In this notebook, we'll walk you through each of the components that are involved in a simple RAG application by building a **Personal Wellness Assistant**.

Imagine having an AI assistant that can answer your health and wellness questions based on a curated knowledge base - that's exactly what we'll build here. We won't be leveraging any fancy tools, just the OpenAI Python SDK, Numpy, and some classic Python.

> NOTE: This was done with Python 3.12.3.

> NOTE: There might be [compatibility issues](https://github.com/wandb/wandb/issues/7683) if you're on NVIDIA driver >552.44 As an interim solution - you can rollback your drivers to the 552.44.

## Table of Contents:

- Task 1: Imports and Utilities
- Task 2: Documents (Loading our Wellness Knowledge Base)
- Task 3: Embeddings and Vectors
- Task 4: Prompts
- Task 5: Retrieval Augmented Generation (Building the Wellness Assistant)
  - 🚧 Activity #1: Enhance Your Wellness Assistant

Let's look at a rather complicated looking visual representation of a basic RAG application.

<img src="https://i.imgur.com/vD8b016.png" />

## Task 1: Imports and Utilities

We're just doing some imports and enabling `async` to work within the Jupyter environment here, nothing too crazy!

In [16]:
from aimakerspace.text_utils import TextFileLoader, CharacterTextSplitter
from aimakerspace.vectordatabase import VectorDatabase
import asyncio

In [17]:
import nest_asyncio
nest_asyncio.apply()

## Task 2: Documents

We'll be concerning ourselves with this part of the flow in the following section:

<img src="https://i.imgur.com/jTm9gjk.png" />

### Loading Source Documents

So, first things first, we need some documents to work with.

While we could work directly with the `.txt` files (or whatever file-types you wanted to extend this to) we can instead do some batch processing of those documents at the beginning in order to store them in a more machine compatible format.

In this case, we're going to parse our text file into a single document in memory.

Let's look at the relevant bits of the `TextFileLoader` class:

```python
def load_file(self):
        with open(self.path, "r", encoding=self.encoding) as f:
            self.documents.append(f.read())
```

We're simply loading the document using the built in `open` method, and storing that output in our `self.documents` list.

> NOTE: We're using a comprehensive Health & Wellness Guide as our sample data. This content covers exercise, nutrition, sleep, stress management, and healthy habits - perfect for building a personal wellness assistant!

In [18]:
text_loader = TextFileLoader("data/HealthWellnessGuide.txt")
documents = text_loader.load_documents()
len(documents)

1

In [19]:
print(documents[0][:100])
import os
print("API key set:", bool(os.environ.get("OPENAI_API_KEY")))

The Personal Wellness Guide
A Comprehensive Resource for Health and Well-being

PART 1: EXERCISE AND
API key set: True


### Splitting Text Into Chunks

As we can see, there is one massive document.

We'll want to chunk the document into smaller parts so it's easier to pass the most relevant snippets to the LLM.

There is no fixed way to split/chunk documents - and you'll need to rely on some intuition as well as knowing your data *very* well in order to build the most robust system.

For this toy example, we'll just split blindly on length.

>There's an opportunity to clear up some terminology here, for this course we will stick to the following:
>
>- "source documents" : The `.txt`, `.pdf`, `.html`, ..., files that make up the files and information we start with in its raw format
>- "document(s)" : single (or more) text object(s)
>- "corpus" : the combination of all of our documents

As you can imagine (though it's not specifically true in this toy example) the idea of splitting documents is to break them into manageable sized chunks that retain the most relevant local context.

In [20]:
text_splitter = CharacterTextSplitter()
split_documents = text_splitter.split_texts(documents)
len(split_documents)

21

Let's take a look at some of the documents we've managed to split.

In [21]:
split_documents[0:1]

['The Personal Wellness Guide\nA Comprehensive Resource for Health and Well-being\n\nPART 1: EXERCISE AND MOVEMENT\n\nChapter 1: Understanding Exercise Basics\n\nExercise is one of the most important things you can do for your health. Regular physical activity can improve your brain health, help manage weight, reduce the risk of disease, strengthen bones and muscles, and improve your ability to do everyday activities.\n\nThe four main types of exercise are aerobic (cardio), strength training, flexibility, and balance exercises. A well-rounded fitness routine includes all four types. Adults should aim for at least 150 minutes of moderate-intensity aerobic activity per week, along with muscle-strengthening activities on 2 or more days per week.\n\nChapter 2: Exercises for Common Problems\n\nLower Back Pain Relief\nLower back pain affects approximately 80% of adults at some point in their lives. Gentle stretching and strengthening exercises can help alleviate discomfort and prevent future

## Task 3: Embeddings and Vectors

Next, we have to convert our corpus into a "machine readable" format as we explored in the Embedding Primer notebook.

Today, we're going to talk about the actual process of creating, and then storing, these embeddings, and how we can leverage that to intelligently add context to our queries.

### OpenAI API Key

In order to access OpenAI's APIs, we'll need to provide our OpenAI API Key!

You can work through the folder "OpenAI API Key Setup" for more information on this process if you don't already have an API Key!

In [ ]:
import os
import openai
from getpass import getpass

openai.api_key = getpass("OpenAI API Key: ")
os.environ["OPENAI_API_KEY"] = openai.api_key

print("API key set:", bool(os.environ.get("OPENAI_API_KEY")))

API key set: True


### Vector Database

Let's set up our vector database to hold all our documents and their embeddings!

While this is all baked into 1 call - we can look at some of the code that powers this process to get a better understanding:

Let's look at our `VectorDatabase().__init__()`:

```python
def __init__(self, embedding_model: EmbeddingModel = None):
        self.vectors = defaultdict(np.array)
        self.embedding_model = embedding_model or EmbeddingModel()
```

As you can see - our vectors are merely stored as a dictionary of `np.array` objects.

Secondly, our `VectorDatabase()` has a default `EmbeddingModel()` which is a wrapper for OpenAI's `text-embedding-3-small` model.

> **Quick Info About `text-embedding-3-small`**:
> - It has a context window of **8191** tokens
> - It returns vectors with dimension **1536**

#### ❓Question #1:

The default embedding dimension of `text-embedding-3-small` is 1536, as noted above. 

1. Is there any way to modify this dimension?
2. What technique does OpenAI use to achieve this?

> NOTE: Check out this [API documentation](https://platform.openai.com/docs/api-reference/embeddings/create) for the answer to question #1.1, and [this paper](https://arxiv.org/abs/2205.13147) for an answer to question #1.2!


##### ✅ Answer:
1. Yes, you can edit integer in the embedding vector object
2. OpenAI uses Matryoshka Representation Learning (MRL) to modify an embedding vector’s effective dimension

We can call the `async_get_embeddings` method of our `EmbeddingModel()` on a list of `str` and receive a list of `float` back!

```python
async def async_get_embeddings(self, list_of_text: List[str]) -> List[List[float]]:
        return await aget_embeddings(
            list_of_text=list_of_text, engine=self.embeddings_model_name
        )
```

We cast those to `np.array` when we build our `VectorDatabase()`:

```python
async def abuild_from_list(self, list_of_text: List[str]) -> "VectorDatabase":
        embeddings = await self.embedding_model.async_get_embeddings(list_of_text)
        for text, embedding in zip(list_of_text, embeddings):
            self.insert(text, np.array(embedding))
        return self
```

And that's all we need to do!

In [22]:
vector_db = VectorDatabase()
vector_db = asyncio.run(vector_db.abuild_from_list(split_documents))

#### ❓Question #2:

What are the benefits of using an `async` approach to collecting our embeddings?

> NOTE: Determining the core difference between `async` and `sync` will be useful! If you get stuck - ask ChatGPT!

##### ✅ Answer:
compared to sync, improves the user experience because will allow the app to respond quicker and make overall UI more responsive. Also, makes the app more scalable as user base grows

So, to review what we've done so far in natural language:

1. We load source documents
2. We split those source documents into smaller chunks (documents)
3. We send each of those documents to the `text-embedding-3-small` OpenAI API endpoint
4. We store each of the text representations with the vector representations as keys/values in a dictionary

### Semantic Similarity

The next step is to be able to query our `VectorDatabase()` with a `str` and have it return to us vectors and text that is most relevant from our corpus.

We're going to use the following process to achieve this in our toy example:

1. We need to embed our query with the same `EmbeddingModel()` as we used to construct our `VectorDatabase()`
2. We loop through every vector in our `VectorDatabase()` and use a distance measure to compare how related they are
3. We return a list of the top `k` closest vectors, with their text representations

There's some very heavy optimization that can be done at each of these steps - but let's just focus on the basic pattern in this notebook.

> We are using [cosine similarity](https://www.engati.com/glossary/cosine-similarity) as a distance metric in this example - but there are many many distance metrics you could use - like [these](https://flavien-vidal.medium.com/similarity-distances-for-natural-language-processing-16f63cd5ba55)

> We are using a rather inefficient way of calculating relative distance between the query vector and all other vectors - there are more advanced approaches that are much more efficient, like [ANN](https://towardsdatascience.com/comprehensive-guide-to-approximate-nearest-neighbors-algorithms-8b94f057d6b6)

In [26]:
vector_db.search_by_text("What exercises help with lower back pain?", k=3)

[(' Relief\nLower back pain affects approximately 80% of adults at some point in their lives. Gentle stretching and strengthening exercises can help alleviate discomfort and prevent future episodes.\n\nRecommended exercises for lower back pain include:\n- Cat-Cow Stretch: Start on hands and knees, alternate between arching your back up (cat) and letting it sag down (cow). Do 10-15 repetitions.\n- Bird Dog: From hands and knees, extend opposite arm and leg while keeping your core engaged. Hold for 5 seconds, then switch sides. Do 10 repetitions per side.\n- Partial Crunches: Lie on your back with knees bent, cross arms over chest, tighten stomach muscles and raise shoulders off floor. Hold briefly, then lower. Do 8-12 repetitions.\n- Knee-to-Chest Stretch: Lie on your back, pull one knee toward your chest while keeping the other foot flat. Hold for 15-30 seconds, then switch legs.\n- Pelvic Tilts: Lie on your back with knees bent, flatten your back against the floor by tightening abs an

## Task 4: Prompts

In the following section, we'll be looking at the role of prompts - and how they help us to guide our application in the right direction.

In this notebook, we're going to rely on the idea of "zero-shot in-context learning".

This is a lot of words to say: "We will ask it to perform our desired task in the prompt, and provide no examples."

### XYZRolePrompt

Before we do that, let's stop and think a bit about how OpenAI's chat models work.

We know they have roles - as is indicated in the following API [documentation](https://platform.openai.com/docs/api-reference/chat/create#chat/create-messages)

There are three roles, and they function as follows (taken directly from [OpenAI](https://platform.openai.com/docs/guides/gpt/chat-completions-api)):

- `{"role" : "system"}` : The system message helps set the behavior of the assistant. For example, you can modify the personality of the assistant or provide specific instructions about how it should behave throughout the conversation. However note that the system message is optional and the model’s behavior without a system message is likely to be similar to using a generic message such as "You are a helpful assistant."
- `{"role" : "user"}` : The user messages provide requests or comments for the assistant to respond to.
- `{"role" : "assistant"}` : Assistant messages store previous assistant responses, but can also be written by you to give examples of desired behavior.

The main idea is this:

1. You start with a system message that outlines how the LLM should respond, what kind of behaviours you can expect from it, and more
2. Then, you can provide a few examples in the form of "assistant"/"user" pairs
3. Then, you prompt the model with the true "user" message.

In this example, we'll be forgoing the 2nd step for simplicity's sake.

#### Utility Functions

You'll notice that we're using some utility functions from the `aimakerspace` module - let's take a peek at these and see what they're doing!

##### XYZRolePrompt

Here we have our `system`, `user`, and `assistant` role prompts.

Let's take a peek at what they look like:

```python
class BasePrompt:
    def __init__(self, prompt):
        """
        Initializes the BasePrompt object with a prompt template.

        :param prompt: A string that can contain placeholders within curly braces
        """
        self.prompt = prompt
        self._pattern = re.compile(r"\{([^}]+)\}")

    def format_prompt(self, **kwargs):
        """
        Formats the prompt string using the keyword arguments provided.

        :param kwargs: The values to substitute into the prompt string
        :return: The formatted prompt string
        """
        matches = self._pattern.findall(self.prompt)
        return self.prompt.format(**{match: kwargs.get(match, "") for match in matches})

    def get_input_variables(self):
        """
        Gets the list of input variable names from the prompt string.

        :return: List of input variable names
        """
        return self._pattern.findall(self.prompt)
```

Then we have our `RolePrompt` which laser focuses us on the role pattern found in most API endpoints for LLMs.

```python
class RolePrompt(BasePrompt):
    def __init__(self, prompt, role: str):
        """
        Initializes the RolePrompt object with a prompt template and a role.

        :param prompt: A string that can contain placeholders within curly braces
        :param role: The role for the message ('system', 'user', or 'assistant')
        """
        super().__init__(prompt)
        self.role = role

    def create_message(self, **kwargs):
        """
        Creates a message dictionary with a role and a formatted message.

        :param kwargs: The values to substitute into the prompt string
        :return: Dictionary containing the role and the formatted message
        """
        return {"role": self.role, "content": self.format_prompt(**kwargs)}
```

We'll look at how the `SystemRolePrompt` is constructed to get a better idea of how that extension works:

```python
class SystemRolePrompt(RolePrompt):
    def __init__(self, prompt: str):
        super().__init__(prompt, "system")
```

That pattern is repeated for our `UserRolePrompt` and our `AssistantRolePrompt` as well.

##### ChatOpenAI

Next we have our model, which is converted to a format analogous to libraries like LangChain and LlamaIndex.

Let's take a peek at how that is constructed:

```python
class ChatOpenAI:
    def __init__(self, model_name: str = "gpt-4.1-mini"):
        self.model_name = model_name
        self.openai_api_key = os.getenv("OPENAI_API_KEY")
        if self.openai_api_key is None:
            raise ValueError("OPENAI_API_KEY is not set")

    def run(self, messages, text_only: bool = True):
        if not isinstance(messages, list):
            raise ValueError("messages must be a list")

        openai.api_key = self.openai_api_key
        response = openai.ChatCompletion.create(
            model=self.model_name, messages=messages
        )

        if text_only:
            return response.choices[0].message.content

        return response
```

#### ❓ Question #3:

When calling the OpenAI API - are there any ways we can achieve more reproducible outputs?

> NOTE: Check out [this section](https://platform.openai.com/docs/guides/text-generation/) of the OpenAI documentation for the answer!

##### ✅ Answer:
Yes, you can do things like 
1) provide a seed parameter
2) keep temperature, top_p, and max_tokens parameters identical across runs
3) use a specific model ID


### Creating and Prompting OpenAI's `gpt-4.1-mini`!

Let's tie all these together and use it to prompt `gpt-4.1-mini`!

In [23]:
from aimakerspace.openai_utils.prompts import (
    UserRolePrompt,
    SystemRolePrompt,
    AssistantRolePrompt,
)

from aimakerspace.openai_utils.chatmodel import ChatOpenAI

chat_openai = ChatOpenAI()
user_prompt_template = "{content}"
user_role_prompt = UserRolePrompt(user_prompt_template)
system_prompt_template = (
    "You are an expert in {expertise}, you always answer in a kind way."
)
system_role_prompt = SystemRolePrompt(system_prompt_template)

messages = [
    system_role_prompt.create_message(expertise="Python"),
    user_role_prompt.create_message(
        content="What is the best way to write a loop?"
    ),
]

response = chat_openai.run(messages)

In [24]:
print(response)

Hello! The best way to write a loop in Python depends on what you're trying to achieve, but generally, Python encourages readability and simplicity.

Here are two common types of loops in Python with examples:

1. **For Loop** – when you want to iterate over a sequence or range:

```python
# Loop through a list
fruits = ['apple', 'banana', 'cherry']
for fruit in fruits:
    print(fruit)

# Loop through a range of numbers
for i in range(5):
    print(i)
```

2. **While Loop** – when you want to repeat something as long as a condition is True:

```python
count = 0
while count < 5:
    print(count)
    count += 1
```

For most scenarios involving iteration, a `for` loop is preferred because it is more readable and less error-prone than managing a `while` loop's condition and counters manually.

If you want more specific advice, feel free to share what you’re trying to loop over, and I’ll be happy to help! 😊


## Task 5: Retrieval Augmented Generation

Now we can create a RAG prompt - which will help our system behave in a way that makes sense!

There is much you could do here, many tweaks and improvements to be made!

In [27]:
RAG_SYSTEM_TEMPLATE = """You are a helpful personal wellness assistant that answers health and wellness questions based strictly on provided context.

Instructions:
- Only answer questions using information from the provided context
- If the context doesn't contain relevant information, respond with "I don't have information about that in my wellness knowledge base"
- Be accurate and cite specific parts of the context when possible
- Keep responses {response_style} and {response_length}
- Only use the provided context. Do not use external knowledge.
- Include a gentle reminder that users should consult healthcare professionals for medical advice when appropriate
- Only provide answers when you are confident the context supports your response."""

RAG_USER_TEMPLATE = """Context Information:
{context}

Number of relevant sources found: {context_count}
{similarity_scores}

Question: {user_query}

Please provide your answer based solely on the context above."""

rag_system_prompt = SystemRolePrompt(
    RAG_SYSTEM_TEMPLATE,
    strict=True,
    defaults={
        "response_style": "concise",
        "response_length": "brief"
    }
)

rag_user_prompt = UserRolePrompt(
    RAG_USER_TEMPLATE,
    strict=True,
    defaults={
        "context_count": "",
        "similarity_scores": ""
    }
)

Now we can create our pipeline!

In [41]:
class RetrievalAugmentedQAPipeline:
    def __init__(self, llm: ChatOpenAI, vector_db_retriever: VectorDatabase, 
                 response_style: str = "detailed", include_scores: bool = False) -> None:
        self.llm = llm
        self.vector_db_retriever = vector_db_retriever
        self.response_style = response_style
        self.include_scores = include_scores

    def run_pipeline(self, user_query: str, k: int = 4, **system_kwargs) -> dict:
        # Retrieve relevant contexts
        context_list = self.vector_db_retriever.search_by_text(user_query, k=k)
        
        context_prompt = ""
        similarity_scores = []
        
        for i, (context, score) in enumerate(context_list, 1):
            context_prompt += f"[Source {i}]: {context}\n\n"
            similarity_scores.append(f"Source {i}: {score:.3f}")
        
        # Create system message with parameters
        system_params = {
            "response_style": self.response_style,
            "response_length": system_kwargs.get("response_length", "detailed")
        }
        
        formatted_system_prompt = rag_system_prompt.create_message(**system_params)
        
        user_params = {
            "user_query": user_query,
            "context": context_prompt.strip(),
            "context_count": len(context_list),
            "similarity_scores": f"Relevance scores: {', '.join(similarity_scores)}" if self.include_scores else ""
        }
        
        formatted_user_prompt = rag_user_prompt.create_message(**user_params)

        return {
            "response": self.llm.run([formatted_system_prompt, formatted_user_prompt]), 
            "context": context_list,
            "context_count": len(context_list),
            "similarity_scores": similarity_scores if self.include_scores else None,
            "prompts_used": {
                "system": formatted_system_prompt,
                "user": formatted_user_prompt
            }
        }

In [38]:
rag_pipeline = RetrievalAugmentedQAPipeline(
    vector_db_retriever=vector_db,
    llm=chat_openai,
    response_style="detailed",
    include_scores=True
)

result = rag_pipeline.run_pipeline(
    "What are some natural remedies for improving sleep quality?",
    k=3,
    response_length="comprehensive", 
    include_warnings=True,
    confidence_required=True
)

print(f"Response: {result['response']}")
print(f"\nContext Count: {result['context_count']}")
print(f"Similarity Scores: {result['similarity_scores']}")

Response: Based on the provided context, some natural remedies for improving sleep quality include:

1. Cognitive Behavioral Therapy for Insomnia (CBT-I) – a structured program that helps address the thoughts and behaviors that prevent good sleep. [Source 1: Chapter 9]

2. Relaxation techniques such as progressive muscle relaxation – helps reduce physical tension and calm the mind before bedtime. [Source 1: Chapter 9; Source 3]

3. Herbal teas like chamomile or valerian root – these are traditionally used to promote relaxation and sleepiness. [Source 1: Chapter 9]

4. Magnesium supplements – may support sleep, but it's advised to consult a healthcare provider before use. [Source 1: Chapter 9]

5. Meditation – helps in calming the mind and reducing stress to facilitate better sleep. [Source 1: Chapter 9]

Additionally, improving sleep hygiene supports better sleep quality naturally, such as:

- Avoiding caffeine after 2 PM. [Source 1; Source 2; Source 3]
- Maintaining a consistent sleep

#### ❓ Question #4:

What prompting strategies could you use to make the LLM have a more thoughtful, detailed response?

What is that strategy called?

> NOTE: You can look through our [OpenAI Responses API](https://colab.research.google.com/drive/14SCfRnp39N7aoOx8ZxadWb0hAqk4lQdL?usp=sharing) notebook for an answer to this question if you get stuck!

##### ✅ Answer:
1. Chain-of-thought prompting - instruct the model to “think step by step” or “show your reasoning”
2. Few-shot prompting - provide several reasoning examples that demonstrate the kind of detailed reasoning you want, then give your new query
3. Reflection prompting - after an initial answer, prompt the model to review and improve its own response
4. Structured output prompting - specify a structured format that requires depth, e.g., “Respond with: (1) Overview, (2) Detailed step-by-step reasoning, (3) Edge cases, (4) Final recommendation.”

### 🏗️ Activity #1:

Enhance your Personal Wellness Assistant in some way! 

Suggestions are: 

- **PDF Support**: Allow it to ingest wellness PDFs (meal plans, workout guides, medical information sheets)
- **New Distance Metric**: Implement a different similarity measure - does it improve retrieval for health queries?
- **Metadata Support**: Add metadata like topic categories (exercise, nutrition, sleep) or difficulty levels to help filter results
- **Different Embedding Model**: Try a different embedding model - does domain-specific tuning help for health content?
- **Multi-Source Ingestion**: Add the capability to ingest content from YouTube health videos, podcasts, or health blogs

While these are suggestions, you should feel free to make whatever augmentations you desire! Think about what features would make this wellness assistant most useful for your personal health journey.

When you're finished making the augments to your RAG application - vibe check it against the old one - see if you can "feel the improvement"!

> NOTE: These additions might require you to work within the `aimakerspace` library - that's expected!

> NOTE: If you're not sure where to start - ask Cursor (CMD/CTRL+L) to guide you through the changes!

In [ ]:
### YOUR CODE HERE

## 🏋️ MuscleWiki Integration

Adding physical fitness content to our wellness assistant using the [MuscleWiki API](https://api.musclewiki.com/documentation).

This integration will allow our RAG pipeline to recommend exercises alongside mental health and wellness advice.
### Step 1: Set Up RapidAPI Key

First, we need to securely store our RapidAPI key for accessing the MuscleWiki API.

In [30]:
import os
import requests
from getpass import getpass

# Securely input your RapidAPI key
rapidapi_key = getpass("RapidAPI Key: ")
os.environ["RAPIDAPI_KEY"] = rapidapi_key

print("RapidAPI key set:", bool(os.environ.get("RAPIDAPI_KEY")))

RapidAPI key set: True


### Step 2: Test API Connection

Let's verify our API connection is working by calling the health endpoint.

In [31]:
# MuscleWiki API configuration
MUSCLEWIKI_HOST = "musclewiki-api.p.rapidapi.com"
MUSCLEWIKI_BASE_URL = f"https://{MUSCLEWIKI_HOST}"

headers = {
    "X-RapidAPI-Key": os.environ.get("RAPIDAPI_KEY"),
    "X-RapidAPI-Host": MUSCLEWIKI_HOST
}

# Test the connection
response = requests.get(f"{MUSCLEWIKI_BASE_URL}/health", headers=headers)

if response.status_code == 200:
    print("✅ API Connection Successful!")
    print(response.json())
else:
    print(f"❌ API Connection Failed: {response.status_code}")
    print(response.text)

✅ API Connection Successful!
{'status': 'healthy', 'version': '2.0.0', 'exercises_loaded': 1734}


### Step 3: Fetch Sample Exercises

Let's fetch a few exercises to see the data structure we'll be working with.

In [6]:
# Fetch some bodyweight exercises (good for stress relief)
response = requests.get(
    f"{MUSCLEWIKI_BASE_URL}/exercises",
    headers=headers,
    params={
        "category": "bodyweight",
        "difficulty": "novice",
        "limit": 5
    }
)

if response.status_code == 200:
    data = response.json()
    
    # API returns: {"total": N, "results": [...]}
    exercises = data.get("results", [])
    
    print(f"Found {len(exercises)} exercises (out of {data.get('total', '?')} total):\n")
    for ex in exercises:
        print(f"- {ex['name']} (ID: {ex['id']})")
else:
    print(f"Error: {response.status_code}")
    print(response.text)

Found 5 exercises (out of 63 total):

- Bench Dips (ID: 13)
- Glute Bridge (ID: 23)
- Crunches (ID: 29)
- Laying Leg Raises (ID: 30)
- Side Lunges (ID: 150)


In [7]:
# Get detailed info for the first exercise (including steps for RAG)
if exercises:
    exercise_id = exercises[0]['id']
    detail_response = requests.get(
        f"{MUSCLEWIKI_BASE_URL}/exercises/{exercise_id}",
        headers=headers
    )
    
    if detail_response.status_code == 200:
        exercise_detail = detail_response.json()
        print(f"Exercise: {exercise_detail['name']}")
        print(f"Category: {exercise_detail.get('category', 'N/A')}")
        print(f"Difficulty: {exercise_detail.get('difficulty', 'N/A')}")
        print(f"Target Muscles: {exercise_detail.get('primary_muscles', [])}")
        print(f"\nInstructions:")
        for i, step in enumerate(exercise_detail.get('steps', []), 1):
            print(f"  {i}. {step}")

Exercise: Bench Dips
Category: Bodyweight
Difficulty: Novice
Target Muscles: ['Triceps']

Instructions:
  1. Grip the edge of the bench with your hands, Keep your feet together and legs straight.
  2. Lower your body straight down.
  3. Slowly press back up to the starting point.
  4. TIP: Make this harder by raising your feet off the floor and adding weight.


### Next Steps

Now that the API connection is working, the next steps are:

1. **Create `MuscleWikiLoader` class** - Following the `TextFileLoader` pattern
2. **Map mental health states to exercises** - Create curated collections
3. **Format exercises for RAG** - Convert to embeddable documents with mental health context
4. **Add to vector database** - Integrate with existing pipeline
5. **Update prompts** - Enable holistic wellness responses

See `MUSCLEWIKI_INTEGRATION.md` for the full implementation plan.

### Step 4: Create MuscleWikiLoader Class

Following the `TextFileLoader` pattern, we'll create a loader that:
1. Fetches exercises from the MuscleWiki API
2. Formats them as text documents with mental health context
3. Returns documents ready for embedding

In [10]:
from typing import List, Dict, Optional

# Mental health to exercise type mappings
MENTAL_HEALTH_EXERCISE_MAP = {
    "stress_relief": {
        "filters": {"category": "bodyweight", "difficulty": "novice"},
        "context": "This exercise is helpful for stress relief and relaxation. Low-intensity movement can help reduce cortisol levels and calm the nervous system."
    },
    "energy_boost": {
        "filters": {"mechanic": "compound", "difficulty": "intermediate"},
        "context": "This exercise is great for boosting energy and improving mood. Compound movements increase blood flow and release endorphins."
    },
    "anxiety_reduction": {
        "filters": {"category": "bodyweight", "difficulty": "novice"},
        "context": "This exercise can help reduce anxiety. Focused, controlled movements encourage mindfulness and help ground you in the present moment."
    },
    "mood_improvement": {
        "filters": {"category": "bodyweight", "mechanic": "compound"},
        "context": "This exercise supports mood improvement. Physical activity stimulates the release of serotonin and dopamine, natural mood elevators."
    },
    "tension_release": {
        "filters": {"category": "bodyweight"},
        "context": "This exercise helps release physical tension. When we're stressed, muscles tighten - movement helps release that stored tension."
    }
}


class MuscleWikiLoader:
    """
    Loader for MuscleWiki API that follows the TextFileLoader pattern.
    Fetches exercises and formats them as documents for RAG.
    """
    
    def __init__(self, api_key: str, host: str = "musclewiki-api.p.rapidapi.com"):
        self.documents = []
        self.headers = {
            "X-RapidAPI-Key": api_key,
            "X-RapidAPI-Host": host
        }
        self.base_url = f"https://{host}"
    
    def fetch_exercises(
        self, 
        category: Optional[str] = None,
        difficulty: Optional[str] = None,
        mechanic: Optional[str] = None,
        muscles: Optional[str] = None,
        limit: int = 20
    ) -> List[Dict]:
        """Fetch exercises from MuscleWiki API with optional filters."""
        params = {"limit": limit}
        if category:
            params["category"] = category
        if difficulty:
            params["difficulty"] = difficulty
        if mechanic:
            params["mechanic"] = mechanic
        if muscles:
            params["muscles"] = muscles
            
        response = requests.get(
            f"{self.base_url}/exercises",
            headers=self.headers,
            params=params
        )
        
        if response.status_code == 200:
            # API returns {"total": N, "results": [...]}
            data = response.json()
            return data.get("results", [])
        else:
            print(f"Error fetching exercises: {response.status_code}")
            return []
    
    def get_exercise_details(self, exercise_id: int) -> Optional[Dict]:
        """Get detailed information for a specific exercise."""
        response = requests.get(
            f"{self.base_url}/exercises/{exercise_id}",
            headers=self.headers
        )
        
        if response.status_code == 200:
            return response.json()
        return None
    
    def format_exercise_as_document(
        self, 
        exercise: Dict, 
        mental_health_context: str = ""
    ) -> str:
        """Convert exercise data to a text document for embedding."""
        # Get detailed info if we only have minimal data
        if 'steps' not in exercise:
            exercise = self.get_exercise_details(exercise['id']) or exercise
        
        # Build the document text
        doc_parts = [
            f"Exercise: {exercise.get('name', 'Unknown')}",
            f"Difficulty: {exercise.get('difficulty', 'Not specified')}",
            f"Equipment: {exercise.get('category', 'Not specified')}",
            f"Target Muscles: {', '.join(exercise.get('primary_muscles', ['Not specified']))}",
        ]
        
        # Add mechanic type if available
        if exercise.get('mechanic'):
            doc_parts.append(f"Type: {exercise['mechanic']}")
        
        # Add instructions
        steps = exercise.get('steps', [])
        if steps:
            doc_parts.append("\nInstructions:")
            for i, step in enumerate(steps, 1):
                doc_parts.append(f"{i}. {step}")
        
        # Add mental health context
        if mental_health_context:
            doc_parts.append(f"\nMental Health Benefit: {mental_health_context}")
        
        return "\n".join(doc_parts)
    
    def load_for_mental_health(self, category: str, limit: int = 10) -> None:
        """Load exercises for a specific mental health category."""
        if category not in MENTAL_HEALTH_EXERCISE_MAP:
            print(f"Unknown category: {category}")
            return
        
        config = MENTAL_HEALTH_EXERCISE_MAP[category]
        exercises = self.fetch_exercises(**config["filters"], limit=limit)
        
        for exercise in exercises:
            doc = self.format_exercise_as_document(
                exercise, 
                mental_health_context=config["context"]
            )
            self.documents.append(doc)
        
        print(f"Loaded {len(exercises)} exercises for {category}")
    
    def load_all_categories(self, limit_per_category: int = 5) -> None:
        """Load exercises for all mental health categories."""
        for category in MENTAL_HEALTH_EXERCISE_MAP:
            self.load_for_mental_health(category, limit=limit_per_category)
    
    def load_documents(self) -> List[str]:
        """Return loaded documents (matches TextFileLoader interface)."""
        return self.documents


print("✅ MuscleWikiLoader class defined!")

✅ MuscleWikiLoader class defined!


### Step 5: Load Exercises for Mental Health Categories

Let's load exercises for each mental health category and see what the documents look like.

In [12]:
# Create the loader with our API key
muscle_loader = MuscleWikiLoader(api_key=os.environ.get("RAPIDAPI_KEY"))

# Load exercises for all mental health categories (5 per category)
muscle_loader.load_all_categories(limit_per_category=5)

# Get the documents
exercise_documents = muscle_loader.load_documents()
print(f"\n📊 Total exercise documents loaded: {len(exercise_documents)}")

Loaded 5 exercises for stress_relief
Loaded 5 exercises for energy_boost
Loaded 5 exercises for anxiety_reduction
Loaded 5 exercises for mood_improvement
Loaded 5 exercises for tension_release

📊 Total exercise documents loaded: 25


In [13]:
# Preview a sample document
print("📄 Sample Exercise Document:\n")
print("=" * 50)
print(exercise_documents[0])
print("=" * 50)

📄 Sample Exercise Document:

Exercise: Bench Dips
Difficulty: Novice
Equipment: Bodyweight
Target Muscles: Triceps
Type: Compound

Instructions:
1. Grip the edge of the bench with your hands, Keep your feet together and legs straight.
2. Lower your body straight down.
3. Slowly press back up to the starting point.
4. TIP: Make this harder by raising your feet off the floor and adding weight.

Mental Health Benefit: This exercise is helpful for stress relief and relaxation. Low-intensity movement can help reduce cortisol levels and calm the nervous system.


### Step 6: Add Exercises to Vector Database

Now we'll chunk the exercise documents and add them to a new vector database that includes both our wellness guide and the MuscleWiki exercises.

In [32]:
# Combine wellness guide documents with exercise documents
combined_documents = documents + exercise_documents  # 'documents' from earlier cell

print(f"Wellness Guide documents: {len(documents)}")
print(f"Exercise documents: {len(exercise_documents)}")
print(f"Combined total: {len(combined_documents)}")

# Split the combined documents into chunks
combined_splitter = CharacterTextSplitter()
combined_chunks = combined_splitter.split_texts(combined_documents)

print(f"\n📊 Total chunks after splitting: {len(combined_chunks)}")

Wellness Guide documents: 1
Exercise documents: 25
Combined total: 26

📊 Total chunks after splitting: 46


In [33]:
# Build the enhanced vector database with both wellness and exercise content
print("🔄 Building enhanced vector database (this may take a moment)...")

enhanced_vector_db = VectorDatabase()
enhanced_vector_db = asyncio.run(enhanced_vector_db.abuild_from_list(combined_chunks))

print("✅ Enhanced vector database built!")

🔄 Building enhanced vector database (this may take a moment)...
✅ Enhanced vector database built!


### Step 7: Test the Enhanced RAG Pipeline

Let's test our enhanced wellness assistant with queries that combine mental health and physical fitness.

In [34]:
# Test retrieval with a mental health + fitness query
test_query = "I'm feeling stressed and anxious. What exercises can help me relax?"

# Search the enhanced database
results = enhanced_vector_db.search_by_text(test_query, k=5)

print(f"🔍 Query: {test_query}\n")
print("📚 Retrieved contexts:\n")
for i, (context, score) in enumerate(results, 1):
    print(f"--- Result {i} (Score: {score:.3f}) ---")
    print(context[:300] + "..." if len(context) > 300 else context)
    print()

🔍 Query: I'm feeling stressed and anxious. What exercises can help me relax?

📚 Retrieved contexts:

--- Result 1 (Score: 0.573) ---
gement involves both immediate relief techniques and long-term lifestyle changes.

Immediate stress relief:
- Deep breathing: Inhale for 4 counts, hold for 4, exhale for 4
- Progressive muscle relaxation: Tense and release muscle groups from toes to head
- Grounding technique: Name 5 things you see,...

--- Result 2 (Score: 0.565) ---
Insomnia (CBT-I)
- Relaxation techniques like progressive muscle relaxation
- Herbal teas such as chamomile or valerian root
- Magnesium supplements (consult healthcare provider first)
- Meditation and deep breathing exercises

PART 4: STRESS MANAGEMENT AND MENTAL WELLNESS

Chapter 10: Understanding...

--- Result 3 (Score: 0.502) ---
 media feeds for positivity

APPENDIX: QUICK REFERENCE GUIDES

Emergency Stress Relief (5 minutes or less):
1. Box breathing: 4 counts in, 4 hold, 4 out, 4 hold
2. Cold water on wrists and face

In [39]:
# Create enhanced RAG pipeline with the new database
enhanced_rag_pipeline = RetrievalAugmentedQAPipeline(
    vector_db_retriever=enhanced_vector_db,
    llm=chat_openai,
    response_style="detailed",
    include_scores=True
)

# Run a holistic wellness query
result = enhanced_rag_pipeline.run_pipeline(
    "I'm feeling stressed and anxious. What exercises can help me relax?",
    k=5,
    response_length="comprehensive"
)

print("🧘 Enhanced Wellness Assistant Response:\n")
print(result['response'])
print(f"\n📊 Context Count: {result['context_count']}")
print(f"📈 Similarity Scores: {result['similarity_scores']}")

🧘 Enhanced Wellness Assistant Response:

To help relax when feeling stressed and anxious, you can try several exercises outlined in the provided wellness context:

Immediate stress relief exercises:
- Deep breathing: Inhale for 4 counts, hold for 4, then exhale for 4 counts (Source 1 & 2)
- Progressive muscle relaxation: Tense and then release muscle groups starting from toes up to the head (Source 1 & 2)
- Grounding technique: Name 5 things you see, 4 you hear, 3 you feel, 2 you smell, and 1 you taste (Source 1 & 3)
- Take a short walk, preferably in nature (Source 1)
- Listen to calming music (Source 1)

Additional quick stress relief (under 5 minutes):
- Box breathing: 4 counts in, 4 hold, 4 out, 4 hold (Source 3)
- Splash cold water on wrists and face (Source 3)
- Step outside for fresh air (Source 3)

Mindfulness meditation can also reduce stress and anxiety:
- Find a quiet, comfortable place to sit
- Close your eyes and focus on your breath
- Notice when your mind wanders, then g

In [36]:
# Try another query
result2 = enhanced_rag_pipeline.run_pipeline(
    "I've been feeling low energy lately. What can I do to boost my mood?",
    k=5,
    response_length="comprehensive"
)

print("🧘 Enhanced Wellness Assistant Response:\n")
print(result2['response'])

🧘 Enhanced Wellness Assistant Response:

To boost your mood and energy when feeling low, you can try several quick and effective strategies based on the wellness context provided:

1. Quick Energy Boosters (from Source 1):
   - Do 10 jumping jacks to increase circulation and energize your body.
   - Drink a glass of cold water to hydrate and refresh yourself.
   - Step outside for sunlight, which can improve mood and energy.
   - Hold a power pose for 2 minutes to help increase confidence and alertness.
   - Have a healthy snack such as nuts or fruit to provide quick nourishment.
   - Take a brief walk around the block for light physical activity.
   - Listen to upbeat music to elevate your mood.
   - Splash cold water on your face for a quick refreshing boost.

2. Manage Stress (from Source 2 and Source 3):
   - Practice stress management techniques like box breathing (inhale 4 counts, hold 4, exhale 4, hold 4).
   - Take moments of mindfulness (daily wellness checklist in Source 5).
